# Handbook of MRI Pulse Sequences: Section 3.3 Refocusing Pulses

In this notebook we will atempt to understand the basic components of a refocusing pulse.

Note: Select "Run" above to run all cells, or press "shift + enter"

First we need to install required packages. Might take a minute or two:

In [ ]:
!pip install -q numpy matplotlib scipy ipywidgets sycomore

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

import sycomore
from sycomore import units

In [3]:
def get_inhomogeneity(B0=3, inhomogeneity_ppm=0.0):

    # Define the gyromagnetic ratio for protons (42.58 MHz/T)
    gyromagnetic_ratio = 42.58e6  # Hz/T

    freq_rot = gyromagnetic_ratio * B0 # resonant frequency 
    freq = gyromagnetic_ratio * B0 * (1 - inhomogeneity_ppm) 

    delta_freq = (freq - freq_rot)*1e-6 # convert from ppm

    return delta_freq*units.Hz 

In [4]:
def get_species(T1, T2, delta_omega=0*units.Hz):   
    species = sycomore.Species(T1, T2, delta_omega=delta_omega)
    return species

def update_and_record_magnetization(M, record, t, update=None):
    if update is not None: M = update @ M
    record.append([t.convert_to(units.ms), M[:3] / M[3]])
    return M

In [17]:
gyromagnetic_ratio = 42.58e6  * units.Hz / units.T 
duration = 1 * units.ms 
amplitude = 1.174*units.mT/units.m
location = 0.1 * units.mm
gyromagnetic_ratio*amplitude*location

4.99889 [ T^-1 ]

In [43]:
interval = sycomore.TimeInterval(1*units.ms, 20*units.mT/units.m)

In [44]:
interval

In [51]:
T1 = 1000 * units.ms
T2 = 100 * units.ms
species = get_species(T1, T2)
step_size = 1 * units.ms 
gradient_amplitude = 1*units.mT/units.m
idle = sycomore.bloch.time_interval(species, step_size, gradient_amplitude=gradient_amplitude, position=1.0*np.random.randn()*units.mm)

In [52]:
idle.round(2)

array([[ 0.99,  0.04,  0.  ,  0.  ],
       [-0.04,  0.99,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  1.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  1.  ]])

In [91]:
def run_simulation(T1_ms, T2_ms, flip_angle_deg, step_size_ms, n_species, gradient_amplitude_mT_m, position_mm, time_post_refocus):
    
    delta_omega = get_inhomogeneity(B0=3, inhomogeneity_ppm=-0.2)

    T1 = T1_ms * units.ms
    T2 = T2_ms * units.ms
    n_species = int(n_species)

    species_baseline = [get_species(T1, T2, delta_omega=-delta_omega)]
    species_ensemble = species_baseline + [get_species(T1, T2, delta_omega=delta_omega*np.random.randn()) for _ in range(n_species-1)]

    # temporal resolution of experiment 
    step_size = step_size_ms * units.ms 

    # nothing occurs during this time, only dephasing
    idles = [sycomore.bloch.time_interval(species, step_size) for species in species_ensemble]

    # dephasing due to delta_omega and gradient
    gradient_amplitude = gradient_amplitude_mT_m*units.mT/units.m
    grads = [sycomore.bloch.time_interval(species, step_size, gradient_amplitude=gradient_amplitude, position=position_mm*np.random.randn()*units.mm) for species in species_ensemble]

    pulse = sycomore.bloch.pulse(flip_angle_deg * units.deg, phase=np.pi*units.rad)

    flip_angle_refocus_deg = 180
    pulse_refocus = sycomore.bloch.pulse(flip_angle_refocus_deg * units.deg, phase=np.pi*units.rad/2) 

    # initialize the spins
    t = 0 * units.s

    M = [np.array([0, 0, 1, 1]) for _ in range(n_species)]

    records = [[[t.convert_to(units.ms), m[:3] / m[3]]] for m in M]

    for k in range(n_species):
        M[k] = update_and_record_magnetization(M[k], records[k], t, update=pulse)

    # Update and record magnetization for the next 100 steps
    for _ in range(45):
        t += step_size
        for k in range(n_species):
            M[k] = update_and_record_magnetization(M[k], records[k], t, update=idles[k])
        
    grad_tau_pre = t.convert_to(units.ms)
    for _ in range(10):
        t += step_size
        for k in range(n_species):
            M[k] = update_and_record_magnetization(M[k], records[k], t, update=grads[k])

    # Update and record magnetization for the next 100 steps
    for _ in range(45):
        t += step_size
        for k in range(n_species):
            M[k] = update_and_record_magnetization(M[k], records[k], t, update=idles[k])
        
    # apply 180 degree refocusing pulse
    tau = t.convert_to(units.ms)
    for k in range(n_species):
        M[k] = update_and_record_magnetization(M[k], records[k], t, update=pulse_refocus)

    # Update and record magnetization for the next 50 steps
    for _ in range(int(time_post_refocus)):
        t += step_size
        for k in range(n_species):
            M[k] = update_and_record_magnetization(M[k], records[k], t, update=idles[k])

    grad_tau_post = t.convert_to(units.ms)
    for _ in range(10):
        t += step_size
        for k in range(n_species):
            M[k] = update_and_record_magnetization(M[k], records[k], t, update=grads[k])

    # Update and record magnetization for the next 100 steps
    for _ in range(100):
        t += step_size
        for k in range(n_species):
            M[k] = update_and_record_magnetization(M[k], records[k], t, update=idles[k])
            
    magnetization = []
    #magnetization_predicted = []
    for species, record in zip(species_ensemble[::-1], records[::-1]):
        time, magnetization_k = list(zip(*record))
        magnetization_k = np.array(magnetization_k)

        #M_pred = calculate_magnetization_post_refocusing(M_post=magnetization_k[-300], time_ms=time[-300:], tau=tau, species=species)
        #magnetization_predicted.append(M_pred)
        magnetization.append(np.array(magnetization_k))

    magnetization = np.stack(magnetization).mean(axis=0)
    #magnetization_predicted = np.stack(magnetization_predicted).mean(axis=0)

    

    fig, ax = plt.subplots(1,2,figsize=(20,10))
    fontsize = 20
    for label, j in zip(['M_x', 'M_y'], [0, 1]):
        ax[j].plot(time, np.linalg.norm(magnetization_k[:, :2], axis=-1), label="Species k $M_\perp$", color='black', linewidth=4)
        ax[j].plot(time, np.linalg.norm(magnetization[:, :2], axis=-1), label="Species Ensemble $M_\perp$", color='#F25050', linewidth=1)
        #ax[j].plot(time[-300:], np.linalg.norm(magnetization_predicted[:, :2], axis=-1), label="Species Ensemble Pred $M_\perp$", color='blue', linewidth=4, linestyle='--')
        ax[j].plot(time, magnetization_k[:, j], label="Species k $%s$"%(label), color='black', linewidth=1)
        ax[j].plot(time, magnetization[:, j], label="Species Ensemble $%s$"%(label), color='#F25050', linewidth=1, linestyle='--')
        #ax[j].plot(time[-300:], magnetization_predicted[:, j], label="Species Ensemble Pred $%s$"%(label), color='blue', linewidth=1, linestyle='--')
        ax[j].axvline(x=tau, color='black')
        ax[j].axvline(x=tau*2, color='black')
        ax[j].axvline(x=grad_tau_pre, color='red', linestyle='--', alpha=0.5)
        ax[j].axvline(x=grad_tau_post, color='red', linestyle='--', alpha=0.5)
        ax[j].axvline(x=grad_tau_post+step_size_ms*10, color='red', linestyle='--', alpha=0.5)
        ax[j].axvline(x=grad_tau_post+step_size_ms*20, color='red', linestyle='--', alpha=0.5)
        ax[j].text(tau, -0.8, 'TE/2', fontsize=fontsize)
        ax[j].text(tau*2, -0.8, 'TE', fontsize=fontsize)
        ax[j].set_xlabel("Time (ms)", fontsize=fontsize)
        ax[j].set_ylabel("$M/M_0$", fontsize=fontsize)
        ax[j].legend(loc='best', fontsize='large')
        ax[j].set_ylim(-1,1)

    plt.show()

In [92]:
widgets.interact(
    run_simulation,
    T1_ms=widgets.FloatSlider(min=100, max=2000, step=10, value=1000, description='T1 (ms)'),
    T2_ms=widgets.FloatSlider(min=10, max=200, step=1, value=88, description='T2 (ms)'),
    flip_angle_deg=widgets.FloatSlider(min=0, max=180, step=1, value=90, description='FA 1 (deg)'),
    step_size_ms=widgets.FloatSlider(min=0.1, max=10, step=0.1, value=0.7, description='Step size (ms)'),
    n_species=widgets.FloatSlider(min=1, max=1000, step=1, value=1000, description='n spins'), 
    gradient_amplitude_mT_m=widgets.FloatSlider(min=0, max=6, step=0.1, value=0.6, description='Gradient'), 
    position_mm=widgets.FloatSlider(min=0, max=6, step=0.1, value=1.3, description='Position (mm)'), 
    time_post_refocus=widgets.FloatSlider(min=45, max=150, step=1, value=96, description='Time Post')
)

interactive(children=(FloatSlider(value=1000.0, description='T1 (ms)', max=2000.0, min=100.0, step=10.0), Floa…

<function __main__.run_simulation(T1_ms, T2_ms, flip_angle_deg, step_size_ms, n_species, gradient_amplitude_mT_m, position_mm, time_post_refocus)>